In [1]:
import os 
import pandas as pd 
import numpy as np 
import re
import plotly.express as px
from tqdm import tqdm
import json 
import shutil
import scipy
import plotly.graph_objects as go

from structuremap.processing import get_smooth_score
from ptmstruct import overview
from ptmstruct import enrichment
from ptmstruct import significant_ptms
from ptmstruct import ptm_cooccurence

from ptmstruct.structural_comparison import af3_job_creation
from ptmstruct.structural_comparison import af3_processing
from ptmstruct.structural_comparison import preprocessing
from ptmstruct.structural_comparison import structural_comparison_analysis 
from ptmstruct.structural_comparison import structure_comparison

from structuremap.processing import (
    download_alphafold_cif,
    download_alphafold_pae,
    format_alphafold_data,
    annotate_accessibility,
    get_smooth_score,
    annotate_proteins_with_idr_pattern,
    get_extended_flexible_pattern
)

/home/max/ma_thesis/code/code_submission/analysis/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-07-07 10:05:29> netCDF4 is not available. Writing AMBER ncdf files will be slow.


In [2]:
proteins_mod = ['2giv', '2i2z', '2ou2', '2ozu', '2x25', '2x2c', '2ybg', '3qah', '4r5a', '4rj3', '4zwi', '5j8c', '6ba2', '6ba4', '6ct2', '6maj', '6mak', '6oin', '6v8n', '7d0o', '8dd5']
proteins_no_mod = ['12ca', '133l', '1a28', '1a3k', '1a6q', '1a7s', '1a8e', '1a8f', '1a9u', '1ad6', '1ads', '1ae5', '1akz', '1anw', '1ao3', '1ao6', '1atk', '1atz']


### Create pPSE DF for experimental structures

In [3]:
import Bio

def split_structure_df(struct_df):
    last_pos = 0
    break_point = None
    for i, row in struct_df.iterrows():
        if row["position"] < last_pos:
            break_point = i
            break
        last_pos = row["position"]

    if break_point:
        struct_df = struct_df.head(break_point)

    return struct_df

def format_experimental_structural_data(
    directory: str,
    protein_ids: list,
    protein_start_number = 0
) -> pd.DataFrame:
    # adapted from structuremap.processing.format_alphafold_data
    alphafold_annotation_l = []
    protein_number = protein_start_number

    for file in tqdm(sorted(os.listdir(directory))):

        if file.endswith("cif"):
            filepath = os.path.join(directory, file)

            protein_id = re.sub(r'.cif', '', file)

            if ((protein_id in protein_ids) or (len(protein_ids) == 0)):#

                protein_number += 1

                structure = Bio.PDB.MMCIF2Dict.MMCIF2Dict(filepath)

                df = pd.DataFrame({'protein_id': f"{protein_id}_org",
                                   'protein_number': protein_number,
                                   'AA': structure['_atom_site.label_comp_id'],
                                   'position': structure['_atom_site.label_seq_id'],
                                   'quality': structure['_atom_site.B_iso_or_equiv'],
                                   'atom_id': structure['_atom_site.label_atom_id'],
                                   'x_coord': structure['_atom_site.Cartn_x'],
                                   'y_coord': structure['_atom_site.Cartn_y'],
                                   'z_coord': structure['_atom_site.Cartn_z']})

                df = df[df.atom_id.isin(['CA', 'CB', 'C', 'N'])].reset_index(drop=True)

                # Convert coordinate columns to numeric
                df[['x_coord', 'y_coord', 'z_coord']] = df[['x_coord', 'y_coord', 'z_coord']].apply(pd.to_numeric)
                
                df = df[df['position'].apply(lambda x: str(x).isdigit())]
                df['position'] = df['position'].astype(int)

                df = split_structure_df(df)

                # Average duplicates
                grouped = df.groupby(['protein_id', 'protein_number', 'AA', 'position', 'atom_id'], as_index=False)[['x_coord', 'y_coord', 'z_coord']].mean()

                # Pivot so each atom type has its own set of columns
                df_wide = grouped.pivot_table(index=['protein_id', 'protein_number', 'AA', 'position'],
                                            columns='atom_id',
                                            values=['x_coord', 'y_coord', 'z_coord'])

                # Flatten MultiIndex columns
                df_wide.columns = [f'{coord}_{atom.lower()}' for coord, atom in df_wide.columns]
                df_wide = df_wide.reset_index()

                alphafold_annotation_l.append(df_wide)

    alphafold_annotation = pd.concat(alphafold_annotation_l)
    alphafold_annotation = alphafold_annotation.sort_values(
        by=['protein_number', 'position']).reset_index(drop=True)
    return(alphafold_annotation)


def annotate_pPSE_values(alphafold_annotation, pae_dir):
    full_sphere_exposure = annotate_accessibility(
        df=alphafold_annotation,
        max_dist=24,
        max_angle=180,
        error_dir=pae_dir)

    alphafold_accessibility = alphafold_annotation.merge(
        full_sphere_exposure, how='left', on=['protein_id', 'AA', 'position'])

    part_sphere_exposure = annotate_accessibility(
        df=alphafold_annotation,
        max_dist=12,
        max_angle=70,
        error_dir=pae_dir)

    alphafold_accessibility = alphafold_accessibility.merge(
        part_sphere_exposure, how='left', on=['protein_id', 'AA', 'position'])
    
    alphafold_accessibility.rename(columns={'nAA_12_70_nopae': 'nAA_12_70_pae', 'nAA_24_180_nopae': 'nAA_24_180_pae'}, inplace=True)

    alphafold_accessibility['high_acc_5'] = np.where(alphafold_accessibility.nAA_12_70_pae <= 5, 1, 0)
    alphafold_accessibility['low_acc_5'] = np.where(alphafold_accessibility.nAA_12_70_pae > 5, 1, 0)

    return alphafold_accessibility

In [4]:
alphafold_annotation_1 = format_experimental_structural_data(directory = "data/af3_validation/org_cif/no_mod", protein_ids = proteins_no_mod)
alphafold_annotation_2 = format_experimental_structural_data(directory = "data/af3_validation/org_cif/mod", protein_ids = proteins_mod, protein_start_number=19)
alphafold_annotation = pd.concat([alphafold_annotation_1, alphafold_annotation_2])
alphafold_annotation

100%|██████████| 21/21 [00:09<00:00,  2.32it/s]


,protein_id,protein_number,AA,position,x_coord_c,x_coord_ca,x_coord_cb,x_coord_n,y_coord_c,y_coord_ca,y_coord_cb,y_coord_n,z_coord_c,z_coord_ca,z_coord_cb,z_coord_n
0,12ca_org,1,TRP,5,6.786,7.743,6.997,8.519,-2.502,-1.668,-0.917,-0.751,10.667,11.585,12.645,10.738
1,12ca_org,1,GLY,6,5.114,5.499,NaN,6.426,-5.671,-4.587,NaN,-3.670,11.515,10.526,NaN,11.232
2,12ca_org,1,TYR,7,5.388,4.509,2.990,4.901,-9.257,-8.048,-8.298,-6.881,11.444,11.868,11.797,11.020
3,12ca_org,1,GLY,8,8.377,7.433,NaN,6.426,-10.300,-9.925,NaN,-8.966,11.444,10.314,NaN,10.738
4,12ca_org,1,LYS,9,11.218,10.243,11.012,9.262,-10.634,-11.759,-12.969,-11.259,12.504,12.151,11.656,11.162
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5546,8dd5_org,40,ARG,277,23.893,24.037,22.931,24.108,-9.270,-8.215,-8.289,-6.884,23.723,24.771,25.811,24.178
5547,8dd5_org,40,TRP,278,22.689,23.063,22.091,23.242,-8.884,-9.716,-10.855,-8.887,20.258,21.482,21.784,22.630
5548,8dd5_org,40,THR,279,22.855,23.241,24.571,23.401,-9.622,-8.524,-7.837,-9.144,16.806,17.827,17.453,19.145
5549,8dd5_org,40,PRO,280,22.819,21.690,20.487,22.033,-10.890,-10.435,-9.870,-9.355,13.838,14.799,14.039,15.759


In [5]:
alphafold_accessibility = annotate_pPSE_values(alphafold_annotation, None)
alphafold_accessibility_smooth = preprocessing.annotate_IDRs(alphafold_accessibility)
df_org_ppse = preprocessing.annotate_short_IDRs(alphafold_accessibility_smooth)

  0%|          | 0/39 [00:00<?, ?it/s]

100%|██████████| 39/39 [00:01<00:00, 34.10it/s]


### Create Af3 Jobs for experimental structures

In [6]:
three_to_one = {
    'ALA': 'A', 'ARG': 'R', 'ASN': 'N', 'ASP': 'D',
    'CYS': 'C', 'GLN': 'Q', 'GLU': 'E', 'GLY': 'G',
    'HIS': 'H', 'ILE': 'I', 'LEU': 'L', 'LYS': 'K',
    'MET': 'M', 'PHE': 'F', 'PRO': 'P', 'SER': 'S',
    'THR': 'T', 'TRP': 'W', 'TYR': 'Y', 'VAL': 'V',
    'SEC': 'U', 'PYL': 'O', 'ASX': 'B', 'GLX': 'Z',
    'XLE': 'J', 'XAA': 'X', 'ALY': 'K'  # ALY (acetyl-lysine), treated as lysine
}

org_aa_dict = (
    df_org_ppse
    .groupby('protein_id')['AA']
    .apply(list)
    .to_dict()
)

job_requests = []
structure_reference_dict = {"present":{}}

length_comp_dict = {
    "mod": [],
    "no_mod": []
}

for elem in org_aa_dict:
    indexes = [i for i, x in enumerate(org_aa_dict[elem]) if x == 'ALY']
    org_aa_str = ''.join(three_to_one.get(aa, 'X') for aa in org_aa_dict[elem])

    if len(indexes) > 1:
        print("Error, more than one Mod, skipping")
        continue

    name = elem.split("_")[0].upper()

    if len(indexes) == 0:
        print(f"For {elem}: No Mod, just inlcuding original")
        job_requests.append(
            (name, [])
        )
        structure_reference_dict["present"][name] = {'sequence':org_aa_str}
        length_comp_dict["no_mod"].append(len(org_aa_str))
        continue

    job_requests.append(
        (name, [(indexes[0] + 1, "K", "ac")])
    )
    structure_reference_dict["present"][name] = {'sequence':org_aa_str}
    length_comp_dict["mod"].append(len(org_aa_str))

job_dict, name_napping, comparisons = af3_job_creation.create_af3_comparison_jobs(job_requests, structure_reference_dict)
#with open("data/structural_validation_2/af3_comparison_nomod_job.json", "w") as json_file:
#    json.dump(job_dict, json_file, indent=4)

print(name_napping)
print(comparisons) 
    

For 12ca_org: No Mod, just inlcuding original
For 133l_org: No Mod, just inlcuding original
For 1a28_org: No Mod, just inlcuding original
For 1a3k_org: No Mod, just inlcuding original
For 1a6q_org: No Mod, just inlcuding original
For 1a7s_org: No Mod, just inlcuding original
For 1a8e_org: No Mod, just inlcuding original
For 1a8f_org: No Mod, just inlcuding original
For 1a9u_org: No Mod, just inlcuding original
For 1ad6_org: No Mod, just inlcuding original
For 1ads_org: No Mod, just inlcuding original
For 1ae5_org: No Mod, just inlcuding original
For 1akz_org: No Mod, just inlcuding original
For 1anw_org: No Mod, just inlcuding original
For 1ao3_org: No Mod, just inlcuding original
For 1ao6_org: No Mod, just inlcuding original
For 1atk_org: No Mod, just inlcuding original
For 1atz_org: No Mod, just inlcuding original
For 2x2c_org: No Mod, just inlcuding original
{'syscomp_job_12ca': '12CA', 'syscomp_job_133l': '133L', 'syscomp_job_1a28': '1A28', 'syscomp_job_1a3k': '1A3K', 'syscomp_job_

In [8]:
print("Median length mod ", np.median(np.array(length_comp_dict["mod"])))
print("Median length no_mod ", np.median(np.array(length_comp_dict["no_mod"])))

Median length mod  270.0
Median length no_mod  223.0


### Process Af3 results

In [7]:
cif_dir, pae_dir, selected_proteins, plddts_dict = af3_processing.load_af3_data(
    alphafold_path = "data/af3_validation/af3/",
    data_dir = "data/af3_validation/",
    af3_name_mapping=name_napping,
    use_present_files = False
)

pred_with_pae = False 
if pred_with_pae:
    df_ppse = preprocessing.perform_annotations(selected_proteins, cif_dir, pae_dir)
else: 
    alphafold_annotation = preprocessing.load_protein_data_from_file(selected_proteins, cif_dir)
    alphafold_accessibility = annotate_pPSE_values(alphafold_annotation, None)
    alphafold_accessibility_smooth = preprocessing.annotate_IDRs(alphafold_accessibility)
    df_ppse = preprocessing.annotate_short_IDRs(alphafold_accessibility_smooth)

Directory 'data/af3_validation/pae' already existed and was deleted.
Directory 'data/af3_validation/cif' already existed and was deleted.


  0%|          | 0/290 [00:00<?, ?it/s]/home/max/ma_thesis/code/code_submission/analysis/ptmstruct/structural_comparison/preprocessing.py:133: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = df.apply(pd.to_numeric, errors='ignore')
  0%|          | 1/290 [00:00<00:38,  7.43it/s]/home/max/ma_thesis/code/code_submission/analysis/ptmstruct/structural_comparison/preprocessing.py:133: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = df.apply(pd.to_numeric, errors='ignore')
  1%|          | 2/290 [00:00<00:38,  7.57it/s]/home/max/ma_thesis/code/code_submission/analysis/ptmstruct/structural_comparison/preprocessing.py:133: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch except

### Extract protein structure

In [8]:
protein_predicted_structure_dict = preprocessing.load_protein_strctures(selected_proteins, cif_dir)
protein_experimental_structure_dict_1 = preprocessing.load_protein_strctures(proteins_no_mod, "data/af3_validation/org_cif/no_mod")
protein_experimental_structure_dict_2 = preprocessing.load_protein_strctures(proteins_mod, "data/af3_validation/org_cif/mod")
protein_experimental_structure_dict = protein_experimental_structure_dict_1 | protein_experimental_structure_dict_2

print(protein_predicted_structure_dict.keys())
print(protein_experimental_structure_dict.keys())

  0%|          | 0/290 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]/home/max/ma_thesis/code/code_submission/analysis/venv/lib/python3.9/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: HG
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
 22%|██▏       | 4/18 [00:00<00:03,  4.64it/s]/home/max/ma_thesis/code/code_submission/analysis/venv/lib/python3.9/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: MN
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
100%|██████████| 21/21 [00:08<00:00,  2.55it/s]

dict_keys(['12CA', '133L', '1A28', '1A3K', '1A6Q', '1A7S', '1A8E', '1A8F', '1A9U', '1AD6', '1ADS', '1AE5', '1AKZ', '1ANW', '1AO3', '1AO6', '1ATK', '1ATZ', '2GIV_ac_K_98', '2GIV', '2I2Z_ac_K_197', '2I2Z', '2OU2_ac_K_98', '2OU2', '2OZU_ac_K_98', '2OZU', '2X25_ac_K_129', '2X25', '2YBG_ac_K_25', '2YBG', '3QAH_ac_K_99', '3QAH', '4R5A_ac_K_153', '4R5A', '4RJ3_ac_K_33', '4RJ3', '4ZWI_ac_K_109', '4ZWI', '5J8C_ac_K_100', '5J8C', '6BA2_ac_K_98', '6BA2', '6BA4_ac_K_98', '6BA4', '6CT2_ac_K_99', '6CT2', '6MAJ_ac_K_98', '6MAJ', '6MAK_ac_K_97', '6MAK', '6OIN_ac_K_98', '6OIN', '6V8N_ac_K_49', '6V8N', '7D0O_ac_K_97', '7D0O', '8DD5_ac_K_97', '8DD5'])
dict_keys(['12ca', '133l', '1a28', '1a3k', '1a6q', '1a7s', '1a8e', '1a8f', '1a9u', '1ad6', '1ads', '1ae5', '1akz', '1anw', '1ao3', '1ao6', '1atk', '1atz', '2giv', '2i2z', '2ou2', '2ozu', '2x25', '2x2c', '2ybg', '3qah', '4r5a', '4rj3', '4zwi', '5j8c', '6ba2', '6ba4', '6ct2', '6maj', '6mak', '6oin', '6v8n', '7d0o', '8dd5'])


## Compute Comparison

In [ ]:
from plotly.subplots import make_subplots

def compute_rmsd_aligned(coords1, coords2_aligned, min_len):
    diff = coords1[:min_len] - coords2_aligned
    rmsd = np.sqrt(np.mean(np.sum(diff**2, axis=1)))
    return rmsd

def compare_structures(structure1, structure2):
    atoms1, coords1, residues1 = structure_comparison.extract_ca_atoms_and_coords(structure1)
    atoms2, coords2, residues2 = structure_comparison.extract_ca_atoms_and_coords(structure2)

    min_len, coords2_aligned = structure_comparison.align_structures(atoms1, atoms2, structure2)

    distances1, distances2 = structure_comparison.compute_consecutive_distances(coords1, coords2_aligned, min_len)
    angles1, angles2, angle_diff = structure_comparison.compute_vectors(structure1, structure2)
    rmsd = compute_rmsd_aligned(coords1, coords2_aligned, min_len)

    return rmsd, np.median(angle_diff) 


def box_plot_consec_diff_results(comparison_results, comparison_label_map, custom_colors):
    consec_data = []
    for comparison, metrics in comparison_results.items():
        display_name = comparison_label_map.get(comparison, comparison)
        for value in metrics["consec_diff"]:
            consec_data.append({"Comparison": display_name, "Value": value})

    df_consec = pd.DataFrame(consec_data)

    fig_consec = px.box(
        df_consec, 
        x="Comparison", 
        y="Value", 
        title="Boxplot of Consecutive Distance MeanErrors between models",
        labels={"Value": "Consecutive Distance Difference"},
        color="Comparison",
        color_discrete_map=custom_colors
    )
    fig_consec.update_layout(
        xaxis_title="Model Comparisons",
        xaxis=dict(showticklabels=False),
        width=1000
    )

    config = {
        'toImageButtonOptions': {
            'format': 'png',
            'filename': "Boxplot of Consecutive Distance MeanErrors between models"
        }
    }

    fig_consec.show(config=config)


def box_plot_angle_diff_results(comparison_results, comparison_label_map, custom_colors):
    angles_data = []
    for comparison, metrics in comparison_results.items():
        display_name = comparison_label_map.get(comparison, comparison)
        for value in metrics["angles"]:
            angles_data.append({"Comparison": display_name, "Value": value})

    df_angles = pd.DataFrame(angles_data)

    fig_angles = px.box(
        df_angles, 
        x="Comparison", 
        y="Value", 
        title="Boxplot of Median of Angle Difference between models",
        labels={"Value": "Angle (degrees)"},
        color="Comparison",
        color_discrete_map=custom_colors
    )

    fig_angles.update_layout(
        xaxis_title="Model Comparisons",
        width=1000,
        xaxis=dict(showticklabels=False)
    )
    config = {
        'toImageButtonOptions': {
            'format': 'png',
            'filename': "Boxplot of Median of Angle Difference between models"
        }
    }
    fig_angles.show(config=config)

def box_plot_diff_results_nocolor(comparison_results, comparison_label_map, metric, yaxis_name, name):
    num_comparisons = len(comparison_results)
    subplot_titles = [comparison_label_map.get(k, k) for k in comparison_results]

    fig = make_subplots(
        rows=1, 
        cols=num_comparisons, 
        subplot_titles=subplot_titles,
        shared_yaxes=True
    )

    for i, (comparison, metrics) in enumerate(comparison_results.items(), start=1):
        values = np.array(metrics[metric])
        median = np.median(values)
        q1 = np.percentile(values, 25)
        q3 = np.percentile(values, 75)
        iqr = q3 - q1
        lower_whisker = np.min(values[values >= q1 - 1.5 * iqr])
        upper_whisker = np.max(values[values <= q3 + 1.5 * iqr])
        min_val = np.min(values)
        max_val = np.max(values)

        print(f"Boxplot for {metric} {comparison}:")
        print(f"  Count: {len(values)}")
        print(f"  Median: {median:.4f}")
        print(f"  Q1 (25th percentile): {q1:.4f}")
        print(f"  Q3 (75th percentile): {q3:.4f}")
        print(f"  IQR: {iqr:.4f}")
        print(f"  Lower Whisker: {lower_whisker:.4f}")
        print(f"  Upper Whisker: {upper_whisker:.4f}")
        print(f"  Min: {min_val:.4f}")
        print(f"  Max: {max_val:.4f}")
        print("")
        fig.add_trace(
            go.Box(
                y=metrics[metric],
                marker_color="black",
                line_color="black",
                boxpoints="outliers",
                name="", 
                showlegend=False
            ),
            row=1, 
            col=i
        )
        fig.update_xaxes(showticklabels=False, row=1, col=i)

    fig.update_layout(
        title_text="", 
        width=250 * num_comparisons,
        height=400,
        margin=dict(t=60, b=60, l=40, r=40),
        yaxis_title=yaxis_name,
        font=dict(
            size=24, 
        )
    )
    for annotation in fig.layout.annotations:
        annotation.font.size = 24

    config = {
        'toImageButtonOptions': {
            'format': 'png',
            'filename': name
        }
    }

    fig.show(config=config)



def box_plot_ppse_comp_results(comparison_results, comparison_label_map, custom_colors):
    ppse_data = []
    for comparison, metrics in comparison_results.items():
        display_name = comparison_label_map.get(comparison, comparison)
        for value in metrics["ppse_me"]:
            ppse_data.append({"Comparison": display_name, "Value": value})

    df_ppse = pd.DataFrame(ppse_data)

    fig_ppse = px.box(
        df_ppse, 
        x="Comparison", 
        y="Value", 
        title="Boxplot of pPSE MeanError between multiple models",
        labels={"Value": "pPSE"},
        color="Comparison",
        color_discrete_map=custom_colors
    )

    fig_ppse.update_layout(
        xaxis_title="Model Comparisons",
        width=1000,
        xaxis=dict(showticklabels=False)
    )
    config = {
        'toImageButtonOptions': {
            'format': 'png',
            'filename': "Boxplot of pPSE MeanError between multiple models"
        }
    }
    fig_ppse.show(config=config)

In [10]:
group_dict, group_std_dict, group_avg_dict = structural_comparison_analysis.extract_ppse_metric_from_df(df_ppse, ppse_name = "nAA_12_70_pae_smooth10")

pred_aa_dict = (
    df_ppse
    .groupby('protein_id')['AA']
    .apply(list)
    .to_dict()
)

org_dict = (
    df_org_ppse
    .groupby('protein_id')['nAA_12_70_pae_smooth10']
    .apply(list)
    .to_dict()
)

org_aa_dict = (
    df_org_ppse
    .groupby('protein_id')['AA']
    .apply(list)
    .to_dict()
)

### Compute Metrics for Experimental (no mod) vs Predicted (no mod) 

In [18]:
comparison_results_exp_nomod = {
    "exp_vs_pre_nomod": {"rmsd": [], "angles": []},
}

comparison_results_exp_nomod_ppse = {
    "exp_vs_pre_nomod": {"ppse_me": [], "low_acc_ppse_me":[], "high_acc_ppse_me":[]},
}

ptms_ppse_dist = []

for pred_key in proteins_no_mod:
    print(elem)
    exp_protein = protein_experimental_structure_dict[pred_key][0][0]
    pre_nomod_protein = protein_predicted_structure_dict[pred_key.upper()][0][0]

    rmsd, mean_error_angles = compare_structures(exp_protein, pre_nomod_protein)
    comparison_results_exp_nomod["exp_vs_pre_nomod"]["rmsd"].append(rmsd)
    comparison_results_exp_nomod["exp_vs_pre_nomod"]["angles"].append(mean_error_angles)



for pred_key in proteins_no_mod:
    org_name = f"{pred_key}_org"

    org_ppse = org_dict[org_name]
    pred_nomod_ppse = group_avg_dict[pred_key.upper()]

    if len(org_ppse) != len(pred_nomod_ppse):
        print("Skipping ", pred_key)
        continue
    mean_error_ppse_exp_vs_pre_nomod = structural_comparison_analysis.mean_error(org_ppse, pred_nomod_ppse)
    comparison_results_exp_nomod_ppse["exp_vs_pre_nomod"]["ppse_me"].append(mean_error_ppse_exp_vs_pre_nomod)


('8DD5_ac_K_97', '8DD5')
✅ Full match: 255 Cα atoms aligned.
('8DD5_ac_K_97', '8DD5')
✅ Full match: 130 Cα atoms aligned.
('8DD5_ac_K_97', '8DD5')
⚠️ Partial match: 251 out of 500 (structure1) and 251 (structure2) Cα atoms aligned.
('8DD5_ac_K_97', '8DD5')
✅ Full match: 137 Cα atoms aligned.
('8DD5_ac_K_97', '8DD5')
✅ Full match: 363 Cα atoms aligned.
('8DD5_ac_K_97', '8DD5')
✅ Full match: 221 Cα atoms aligned.
('8DD5_ac_K_97', '8DD5')
✅ Full match: 329 Cα atoms aligned.
('8DD5_ac_K_97', '8DD5')
✅ Full match: 329 Cα atoms aligned.
('8DD5_ac_K_97', '8DD5')
✅ Full match: 351 Cα atoms aligned.
('8DD5_ac_K_97', '8DD5')
✅ Full match: 185 Cα atoms aligned.
('8DD5_ac_K_97', '8DD5')
✅ Full match: 315 Cα atoms aligned.
('8DD5_ac_K_97', '8DD5')
✅ Full match: 223 Cα atoms aligned.
('8DD5_ac_K_97', '8DD5')
✅ Full match: 223 Cα atoms aligned.
('8DD5_ac_K_97', '8DD5')
⚠️ Partial match: 319 out of 642 (structure1) and 319 (structure2) Cα atoms aligned.
('8DD5_ac_K_97', '8DD5')
⚠️ Partial match: 187 o

### Compute Metrics for Experimental (mod) vs Predicted (mod) vs Predicted (no mod) 

In [19]:
comparison_results_exp_mod = {
    "exp_vs_pre_mod": {"rmsd": [], "angles": []},
    "exp_vs_pre_nomod": {"rmsd": [], "angles": []},
    "pre_mod_vs_pre_nomod": {"rmsd": [], "angles": []},
}

for elem in comparisons:
    print(elem) 
    
    # Retrieve structures
    exp_protein = protein_experimental_structure_dict[elem[1].lower()][0][0]
    pre_mod_protein = protein_predicted_structure_dict[elem[0]][0][0]
    pre_nomod_protein = protein_predicted_structure_dict[elem[1]][0][0]

    # Compare experimental vs predicted (with modification)
    rmsd, mean_error_angles = compare_structures(exp_protein, pre_mod_protein)
    comparison_results_exp_mod["exp_vs_pre_mod"]["rmsd"].append(rmsd)
    comparison_results_exp_mod["exp_vs_pre_mod"]["angles"].append(mean_error_angles)

    # Compare experimental vs predicted (no modification)
    rmsd, mean_error_angles = compare_structures(exp_protein, pre_nomod_protein)
    comparison_results_exp_mod["exp_vs_pre_nomod"]["rmsd"].append(rmsd)
    comparison_results_exp_mod["exp_vs_pre_nomod"]["angles"].append(mean_error_angles)

    # Compare predicted (with modification) vs predicted (no modification)
    rmsd, mean_error_angles = compare_structures(pre_mod_protein, pre_nomod_protein)
    comparison_results_exp_mod["pre_mod_vs_pre_nomod"]["rmsd"].append(rmsd)
    comparison_results_exp_mod["pre_mod_vs_pre_nomod"]["angles"].append(mean_error_angles)

('2GIV_ac_K_98', '2GIV')
✅ Full match: 258 Cα atoms aligned.
✅ Full match: 258 Cα atoms aligned.
✅ Full match: 258 Cα atoms aligned.
('2I2Z_ac_K_197', '2I2Z')
✅ Full match: 582 Cα atoms aligned.
✅ Full match: 582 Cα atoms aligned.
✅ Full match: 582 Cα atoms aligned.
('2OU2_ac_K_98', '2OU2')
✅ Full match: 246 Cα atoms aligned.
✅ Full match: 246 Cα atoms aligned.
✅ Full match: 246 Cα atoms aligned.
('2OZU_ac_K_98', '2OZU')
✅ Full match: 254 Cα atoms aligned.
✅ Full match: 254 Cα atoms aligned.
✅ Full match: 254 Cα atoms aligned.
('2X25_ac_K_129', '2X25')
✅ Full match: 168 Cα atoms aligned.
✅ Full match: 168 Cα atoms aligned.
✅ Full match: 168 Cα atoms aligned.
('2YBG_ac_K_25', '2YBG')
⚠️ Partial match: 192 out of 773 (structure1) and 192 (structure2) Cα atoms aligned.
⚠️ Partial match: 192 out of 773 (structure1) and 192 (structure2) Cα atoms aligned.
✅ Full match: 192 Cα atoms aligned.
('3QAH_ac_K_99', '3QAH')
✅ Full match: 274 Cα atoms aligned.
✅ Full match: 274 Cα atoms aligned.
✅ Ful

In [20]:
comparison_results_exp_mod["exp_nomod_vs_pre_nomod"] = comparison_results_exp_nomod["exp_vs_pre_nomod"]


comparison_label_map = {
        "exp_vs_pre_mod": "(Experimental_with_Mod, Predicted_with_Mod)",
        "exp_vs_pre_nomod": "(Experimental_with_Mod, Predicted_without_Mod)",
        "pre_mod_vs_pre_nomod": "(Predicted_with_Mod, Predicted_without_Mod)",
        "exp_nomod_vs_pre_nomod": "(Experimental_without_Mod, Predicted_without_Mod)",
    }

comparison_label_map = {
        "exp_vs_pre_mod": "EWM vs PWM",
        "exp_vs_pre_nomod": "EWM vs PWoM",
        "pre_mod_vs_pre_nomod": "PWM vs PWoM",
        "exp_nomod_vs_pre_nomod": "EWoM vs PWoM",
    }


custom_colors = {
    "(Experimental_with_Mod, Predicted_with_Mod)": "#3f7f3f",
    "(Experimental_with_Mod, Predicted_without_Mod)": "#75eb75",
    "(Predicted_with_Mod, Predicted_without_Mod)": "purple",
    "(Experimental_without_Mod, Predicted_without_Mod)": "#ffd700" 
}

box_plot_diff_results_nocolor(comparison_results_exp_mod, comparison_label_map, "rmsd", "RMSD (in Å)", "Facet1_Boxplot of rmsd between Models")
box_plot_diff_results_nocolor(comparison_results_exp_mod, comparison_label_map, "angles", "Angle (degrees)", "Facet2_Boxplot of Median of Angle Difference between models")

Boxplot for rmsd exp_vs_pre_mod:
  Count: 20
  Median: 1.0939
  Q1 (25th percentile): 0.8262
  Q3 (75th percentile): 1.3357
  IQR: 0.5095
  Lower Whisker: 0.1793
  Upper Whisker: 1.6306
  Min: 0.1793
  Max: 11.3511

Boxplot for rmsd exp_vs_pre_nomod:
  Count: 20
  Median: 0.9817
  Q1 (25th percentile): 0.7399
  Q3 (75th percentile): 1.5123
  IQR: 0.7724
  Lower Whisker: 0.2231
  Upper Whisker: 2.0446
  Min: 0.2231
  Max: 12.4461

Boxplot for rmsd pre_mod_vs_pre_nomod:
  Count: 20
  Median: 0.5663
  Q1 (25th percentile): 0.3718
  Q3 (75th percentile): 0.9751
  IQR: 0.6033
  Lower Whisker: 0.1120
  Upper Whisker: 1.6308
  Min: 0.1120
  Max: 4.3446

Boxplot for rmsd exp_nomod_vs_pre_nomod:
  Count: 18
  Median: 0.5705
  Q1 (25th percentile): 0.3371
  Q3 (75th percentile): 1.3037
  IQR: 0.9666
  Lower Whisker: 0.2497
  Upper Whisker: 2.3940
  Min: 0.2497
  Max: 6.6381



Boxplot for angles exp_vs_pre_mod:
  Count: 20
  Median: 4.3176
  Q1 (25th percentile): 3.5667
  Q3 (75th percentile): 11.7743
  IQR: 8.2077
  Lower Whisker: 1.8627
  Upper Whisker: 12.0598
  Min: 1.8627
  Max: 97.8154

Boxplot for angles exp_vs_pre_nomod:
  Count: 20
  Median: 76.3289
  Q1 (25th percentile): 73.2580
  Q3 (75th percentile): 83.0093
  IQR: 9.7513
  Lower Whisker: 63.7268
  Upper Whisker: 95.0475
  Min: 4.4601
  Max: 113.8874

Boxplot for angles pre_mod_vs_pre_nomod:
  Count: 20
  Median: 77.4544
  Q1 (25th percentile): 74.9439
  Q3 (75th percentile): 77.8770
  IQR: 2.9331
  Lower Whisker: 74.4702
  Upper Whisker: 78.0308
  Min: 1.6147
  Max: 114.9639

Boxplot for angles exp_nomod_vs_pre_nomod:
  Count: 18
  Median: 4.9914
  Q1 (25th percentile): 3.5570
  Q3 (75th percentile): 5.5447
  IQR: 1.9877
  Lower Whisker: 2.9191
  Upper Whisker: 7.3060
  Min: 2.9191
  Max: 73.8233



In [21]:
comparison_results = {
    "exp_vs_pre_mod": {"ppse_me": [], "low_acc_ppse_me":[], "high_acc_ppse_me":[]},
    "exp_vs_pre_nomod": {"ppse_me": [], "low_acc_ppse_me":[], "high_acc_ppse_me":[]},
    "pre_mod_vs_pre_nomod": {"ppse_me": [], "low_acc_ppse_me":[], "high_acc_ppse_me":[]},
}

ptms_ppse_dist = []

for elem in comparisons:
    org_name = f"{elem[1].lower()}_org"

    mod_index = int(elem[0].split("_")[-1])

    org_ppse = org_dict[org_name]
    pred_mod_ppse = group_avg_dict[elem[0]]
    pred_nomod_ppse = group_avg_dict[elem[1]]

    ptms_ppse_dist.append(pred_nomod_ppse[mod_index])

    if len(org_ppse) != len(pred_mod_ppse):
        print("Skipping ", elem)
        continue

    mean_error_ppse_exp_vs_pre_mod = structural_comparison_analysis.mean_error(org_ppse, pred_mod_ppse)
    comparison_results["exp_vs_pre_mod"]["ppse_me"].append(mean_error_ppse_exp_vs_pre_mod)

    mean_error_ppse_exp_vs_pre_nomod = structural_comparison_analysis.mean_error(org_ppse, pred_nomod_ppse)
    comparison_results["exp_vs_pre_nomod"]["ppse_me"].append(mean_error_ppse_exp_vs_pre_nomod)

    mean_error_ppse_pre_mod_vs_pre_nomod = structural_comparison_analysis.mean_error(pred_mod_ppse, pred_nomod_ppse)
    comparison_results["pre_mod_vs_pre_nomod"]["ppse_me"].append(mean_error_ppse_pre_mod_vs_pre_nomod)

fig = go.Figure()

fig.add_trace(go.Box(
    y=ptms_ppse_dist,
    x=[""] * len(ptms_ppse_dist),
    boxpoints='outliers',
    marker_color='black',
    line_color='black',
))


fig.update_layout(
    title='Box Plot of pPSE (smoothed) distribution of PTMs',
    yaxis_title='pPSE of PTM',
    width=600,  
)

fig.show()
comparison_results["exp_nomod_vs_pre_nomod"] = comparison_results_exp_nomod_ppse["exp_vs_pre_nomod"]

comparison_results_exp_nomod_ppse
comparison_label_map = {
        "exp_vs_pre_mod": "EWM vs PWM",
        "exp_vs_pre_nomod": "EWM vs PWoM",
        "pre_mod_vs_pre_nomod": "PWM vs PWoM",
        "exp_nomod_vs_pre_nomod": "EWoM vs PWoM",
    }


custom_colors = {
    "(Experimental_with_Mod, Predicted_with_Mod)": "#3f7f3f",
    "(Experimental_with_Mod, Predicted_without_Mod)": "#75eb75",
    "(Predicted_with_Mod, Predicted_without_Mod)": "purple",
    "(Experimental_without_Mod, Predicted_without_Mod)": "#ffd700" 
}

box_plot_diff_results_nocolor(comparison_results, comparison_label_map, "ppse_me", "pPSE ME", "Facet3_Boxplot of pPSE MeanError between multiple models")

Boxplot for ppse_me exp_vs_pre_mod:
  Count: 20
  Median: 0.0517
  Q1 (25th percentile): -0.0460
  Q3 (75th percentile): 0.1060
  IQR: 0.1520
  Lower Whisker: -0.2515
  Upper Whisker: 0.2631
  Min: -0.7168
  Max: 1.0575

Boxplot for ppse_me exp_vs_pre_nomod:
  Count: 20
  Median: -0.0385
  Q1 (25th percentile): -0.0931
  Q3 (75th percentile): 0.0598
  IQR: 0.1530
  Lower Whisker: -0.1594
  Upper Whisker: 0.0932
  Min: -0.8068
  Max: 1.1033

Boxplot for ppse_me pre_mod_vs_pre_nomod:
  Count: 20
  Median: -0.0598
  Q1 (25th percentile): -0.0929
  Q3 (75th percentile): -0.0124
  IQR: 0.0805
  Lower Whisker: -0.1998
  Upper Whisker: 0.0458
  Min: -0.6453
  Max: 0.0458

Boxplot for ppse_me exp_nomod_vs_pre_nomod:
  Count: 18
  Median: -0.0445
  Q1 (25th percentile): -0.1166
  Q3 (75th percentile): 0.0166
  IQR: 0.1332
  Lower Whisker: -0.1983
  Upper Whisker: 0.0680
  Min: -0.3372
  Max: 0.4469

